In [5]:
#install spark-nlp
!pip install spark-nlp==3.1.0

     |████████████████████████████████| 44 kB 1.1 MB/s 


In [ ]:
# get pyspark
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [9]:
#load required packages
import os
import numpy as np
import pandas as pd
import sparknlp
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *


In [ ]:
#start spark sessiom
spark = sparknlp.start() 


In [10]:

training_df = pd.read_excel(open('Basis State Industry List 08_15_2021.xlsx', 'rb'),
              sheet_name='Training List')
training_df

,Organization - ID,Organization - Name,Organization - Website,Organization - CB Description,Organization - Company Type,Organization - CB Industries,Organization - Employee Range,Organization - New Industry,Keyword,MetaKeyword,MetaDescription
0,440,Aston Martin Lagonda,https://www.astonmartin.comen-us/,Aston Martin Lagonda Global Holdings manufactu...,Acquirer,"Automotive, Leisure, Manufacturing, Product De...",1001-5000,Automotive - Car Manufacturing,NaN,NaN,NaN
1,898,Fiat Chrysler Auto,https://www.fcagroup.comen-US/Pages/home.aspx,"Fiat Chrysler Automobiles designs, engineers, ...",Acquirer,"Automotive, Industrial Manufacturing, Manufact...",10001+,Automotive - Car Manufacturing,NaN,NaN,NaN
2,930,Ford Motor Company,https://www.ford.com,"Ford Motor Company, an automotive company, dev...",Acquirer,"Apps, Automotive, Manufacturing, Transportation",10001+,Automotive - Car Manufacturing,access,NaN,NaN
3,965,General Motors Company,https://www.gm.com,General Motors is a Detroit-based mutinational...,Acquirer,"Automotive, Enterprise Software, Manufacturing...",10001+,Automotive - Car Manufacturing,access,NaN,NaN
4,296,AARP,http://www.aarp.org,NaN,Acquirer,NaN,NaN,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel, ...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2164,48844,Healthtrust Workforce Solutions,https://healthtrustjobs.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN
2165,48845,Signature Consultants,https://www.sigconsult.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN
2166,48846,Acro Service Corporation,https://www.acrocorp.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN
2167,48847,DISYS,https://www.disys.com/,NaN,Acquirer,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:

training_df.columns=['org_id','org_name','org_website','org_cb_description','org_company_type','org_cb_industry','org_company_size','org_new_industry','keyword','metakeyword','metadesc']
drop_col=['org_id','org_name','org_website','org_company_type','org_cb_industry','org_company_size']
training_df.drop(columns=drop_col,axis=1, inplace=True)
training_df


,org_cb_description,org_new_industry,keyword,metakeyword,metadesc
0,Aston Martin Lagonda Global Holdings manufactu...,Automotive - Car Manufacturing,NaN,NaN,NaN
1,"Fiat Chrysler Automobiles designs, engineers, ...",Automotive - Car Manufacturing,NaN,NaN,NaN
2,"Ford Motor Company, an automotive company, dev...",Automotive - Car Manufacturing,access,NaN,NaN
3,General Motors is a Detroit-based mutinational...,Automotive - Car Manufacturing,access,NaN,NaN
4,NaN,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel, ...",NaN,NaN
...,...,...,...,...,...
2164,NaN,NaN,NaN,NaN,NaN
2165,NaN,NaN,NaN,NaN,NaN
2166,NaN,NaN,NaN,NaN,NaN
2167,NaN,NaN,NaN,NaN,NaN


In [12]:
training_df = training_df[training_df['org_new_industry'].notna()]
training_df

,org_cb_description,org_new_industry,keyword,metakeyword,metadesc
0,Aston Martin Lagonda Global Holdings manufactu...,Automotive - Car Manufacturing,NaN,NaN,NaN
1,"Fiat Chrysler Automobiles designs, engineers, ...",Automotive - Car Manufacturing,NaN,NaN,NaN
2,"Ford Motor Company, an automotive company, dev...",Automotive - Car Manufacturing,access,NaN,NaN
3,General Motors is a Detroit-based mutinational...,Automotive - Car Manufacturing,access,NaN,NaN
4,NaN,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel, ...",NaN,NaN
...,...,...,...,...,...
2120,JetBlue airways to bring humanity back to air ...,Transportation - Airline,NaN,NaN,NaN
2121,Southwest Airlines (NYSE: LUV) continues to di...,Transportation - Airline,NaN,NaN,NaN
2122,United Airlines and United Express operate app...,Transportation - Airline,NaN,NaN,NaN
2123,FedEx Corporation provides customers and busin...,Transportation - Shipping,NaN,NaN,NaN


In [13]:
training_df['description_combined'] = training_df[['org_cb_description', 'keyword', 'metakeyword','metadesc']].fillna('').agg(' '.join, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
training_df.dtypes

org_cb_description      object
org_new_industry        object
keyword                 object
metakeyword             object
metadesc                object
description_combined    object
dtype: object

In [15]:
train_df=training_df.loc[:,['org_new_industry','description_combined']]
train_df

,org_new_industry,description_combined
0,Automotive - Car Manufacturing,Aston Martin Lagonda Global Holdings manufactu...
1,Automotive - Car Manufacturing,"Fiat Chrysler Automobiles designs, engineers, ..."
2,Automotive - Car Manufacturing,"Ford Motor Company, an automotive company, dev..."
3,Automotive - Car Manufacturing,General Motors is a Detroit-based mutinational...
4,"Civic, Non-Profit and Membership Groups","benefits, website, leaving, provider, cancel,..."
...,...,...
2120,Transportation - Airline,JetBlue airways to bring humanity back to air ...
2121,Transportation - Airline,Southwest Airlines (NYSE: LUV) continues to di...
2122,Transportation - Airline,United Airlines and United Express operate app...
2123,Transportation - Shipping,FedEx Corporation provides customers and busin...


--2021-08-27 04:37:23--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-08-27 04:37:24--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-08-27 04:37:24 (27.5 MB/s) - written to stdout [1608/1608]

setup Cola

In [16]:
#start spark session
import sparknlp
spark = sparknlp.start() 


Pretrained models

Text Classification with Classifier with SparkNLP

In [17]:
document = DocumentAssembler()\
    .setInputCol("description_combined")\
    .setOutputCol("document")

In [18]:
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [19]:
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("org_new_industry")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

In [20]:
use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

train_sparkdf=spark.createDataFrame(train_df) 

In [ ]:
use_pipelineModel = use_clf_pipeline.fit(train_sparkdf)

In [ ]:
!cat /root/annotator_logs/ClassifierDLApproach_c570d0becc72.log

Training started - epochs: 5 - learning_rate: 0.005 - batch_size: 64 - training_examples: 2125 - classes: 90
Epoch 0/5 - 0.66s - loss: 152.42564 - acc: 0.057291668 - batches: 34
Epoch 1/5 - 0.44s - loss: 151.63655 - acc: 0.068655305 - batches: 34
Epoch 2/5 - 0.45s - loss: 151.63655 - acc: 0.068655305 - batches: 34
Epoch 3/5 - 0.40s - loss: 151.63655 - acc: 0.068655305 - batches: 34
Epoch 4/5 - 0.40s - loss: 151.63655 - acc: 0.068655305 - batches: 34


Bert Sentence Embedding 

In [ ]:
document = DocumentAssembler()\
    .setInputCol("description_combined")\
    .setOutputCol("document")

In [ ]:
bert_sent=BertSentenceEmbeddings.pretrained("sent_small_bert_L8_512")\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [ ]:
classsifierd2 = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("org_new_industry")\
  .setMaxEpochs(20)\
  .setEnableOutputLogs(True)

In [ ]:
bert_sent_clf_pipeline=Pipeline(
    stages=
    [document,bert_sent,classsifierd2])

In [ ]:
bert_sentPiplelineModel=bert_sent_clf_pipeline.fit(train_sparkdf)

In [ ]:
!cat /root/annotator_logs/ClassifierDLApproach_5546c333482b.log

Training started - epochs: 20 - learning_rate: 0.005 - batch_size: 64 - training_examples: 2125 - classes: 90
Epoch 0/20 - 0.53s - loss: 151.60966 - acc: 0.06767191 - batches: 34
Epoch 1/20 - 0.39s - loss: 151.98505 - acc: 0.07149621 - batches: 34
Epoch 2/20 - 0.40s - loss: 151.63652 - acc: 0.06581439 - batches: 34
Epoch 3/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 4/20 - 0.40s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 5/20 - 0.40s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 6/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 7/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 8/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 9/20 - 0.38s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 10/20 - 0.38s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 11/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - batches: 34
Epoch 12/20 - 0.39s - loss: 151.63655 - acc: 0.06581439 - ba

Using Glove Embeddings

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("description_combined") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
      
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")




lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierd3 = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("org_new_industry")\
      .setMaxEpochs(3)\
      .setEnableOutputLogs(True)
      #.setOutputLogsPath('logs')

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierd3])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
clf_pipelineModel = clf_pipeline.fit(train_sparkdf)

In [ ]:
!cat /root/annotator_logs/ClassifierDLApproach_ad793ec19836.log

Training started - epochs: 3 - learning_rate: 0.005 - batch_size: 64 - training_examples: 2125 - classes: 90
Epoch 0/3 - 0.59s - loss: 150.31068 - acc: 0.053430945 - batches: 34
Epoch 1/3 - 0.53s - loss: 150.63655 - acc: 0.058165792 - batches: 34
Epoch 2/3 - 0.41s - loss: 150.63657 - acc: 0.058165792 - batches: 34


Text preprocessig Spark NLP, Count Vectorizer and Logistic Regression

In [22]:

from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer
document_assembler = DocumentAssembler() \
    .setInputCol("description_combined") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
      
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

finisher = Finisher() \
      .setInputCols(["lemma"]) \
      .setOutputCols(["token_features"]) \
      .setOutputAsArray(True) \
      .setCleanAnnotations(False)

countVectors = CountVectorizer(inputCol="token_features", outputCol="features", vocabSize=10000, minDF=5)
label_stringIdx = StringIndexer(inputCol = "org_new_industry", outputCol = "label")
nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            finisher,
            countVectors,
            label_stringIdx])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [23]:
log_model = nlp_pipeline.fit(train_sparkdf)

processed = log_model.transform(train_sparkdf)

processed.count()

2125

In [24]:
(trainingData, testData) = processed.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1493
Test Dataset Count: 632


In [25]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 32.0) \
    .select("description_combined","org_new_industry","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+------------------------------+------------------------------+-----+----------+
|          description_combined|              org_new_industry|                   probability|label|prediction|
+------------------------------+------------------------------+------------------------------+-----+----------+
| management, software, brav...|  SAAS - Contractor Engagement|[0.05234785735974428,0.0368...| 22.0|      32.0|
| nature, geonavtarget, http...|Civic, Non-Profit and Membe...|[0.03736168911371134,0.0080...| 13.0|      32.0|
| carbon, removal, slide, co...|Energy and Environmental - ...|[0.011745997586741034,0.013...| 32.0|      32.0|
|3Degrees is a provider of c...|Energy and Environmental - ...|[0.004451716914380962,2.745...| 32.0|      32.0|
| carbon, offsets, methane, ...|Energy and Environmental - ...|[3.009146850515165E-4,6.826...| 32.0|      32.0|
+------------------------------+------------------------------+------------------------------+-----+----

In [26]:
#Evaluate multiclass predictions
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

evaluator.evaluate(predictions)

0.2547725718659396

In [38]:
#Accuracy
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()
print(accuracy_score(y_true.label, y_pred.prediction))

0.28955696202531644


In [27]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()
#class labels and counts
y_pred.prediction.value_counts()

0.0     257
2.0      84
1.0      53
3.0      44
4.0      32
5.0      25
7.0      14
6.0      14
11.0     11
9.0      11
13.0     10
16.0     10
10.0      8
19.0      6
14.0      6
27.0      5
20.0      5
32.0      5
24.0      4
48.0      3
36.0      3
23.0      3
25.0      2
22.0      2
15.0      2
50.0      2
47.0      1
44.0      1
58.0      1
30.0      1
43.0      1
69.0      1
65.0      1
29.0      1
56.0      1
26.0      1
60.0      1
Name: prediction, dtype: int64

Logistic Regression & TFIDF 

In [28]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures", numFeatures=10000)

idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

nlp_pipeline_tf = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            finisher,
            hashingTF,
            idf,
            label_stringIdx])

model_tf = nlp_pipeline_tf.fit(train_sparkdf)

processed_tf = model_tf.transform(train_sparkdf)

processed_tf.count()

2125

In [44]:
(trainingData, testData) = processed_tf.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1493
Test Dataset Count: 632


In [45]:

lr_tf = lr.fit(trainingData)

predictions_tf = lr_tf.transform(testData)

predictions_tf.select("description_combined","org_new_industry","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+------------------------------+------------------------------+-----+----------+
|          description_combined|              org_new_industry|                   probability|label|prediction|
+------------------------------+------------------------------+------------------------------+-----+----------+
| sefcu, online, tuesday, we...|Finance/Banking - Credit Union|[0.9911761776840043,3.52518...| 24.0|       0.0|
|Infor provides enterprise s...|SAAS - Leisure, Sports and ...|[0.7210925533063297,2.10307...| 57.0|       0.0|
|MedPro Healthcare Staffing ...|Corporate Services - Consul...|[0.6544045096437753,0.00702...|  0.0|       0.0|
|Modern Health is a mental w...|Leisure, Sports and Recreat...|[0.6129711349705067,0.01035...| 38.0|       0.0|
| created, public, insights,...|Energy and Environmental - ...|[0.5604486377860693,0.00738...| 36.0|       0.0|
| insights, digital, energy,...|Corporate Services - Consul...|[0.557076343128009,0.008085...|  0.0|    

In [40]:
evaluator.evaluate(predictions_tf)

y_true = predictions_tf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_tf.select("prediction")
y_pred = y_pred.toPandas()


print(accuracy_score(y_true.label, y_pred.prediction))

0.2674050632911392


Random Forest

In [46]:
#Random Forest
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)


model_rf = rf.fit(trainingData)
predictions_rf = model_rf.transform(testData)


In [47]:
y_true = predictions_rf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_rf.select("prediction")
y_pred = y_pred.toPandas()
print(accuracy_score(y_true.label, y_pred.prediction))

0.16455696202531644


Sentence Embedding using Glove and Logistic regression

In [35]:
document_assembler = DocumentAssembler() \
      .setInputCol("description_combined") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'cleanTokens'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")

label_stringIdx = StringIndexer(inputCol = "org_new_industry", outputCol = "label")


nlp_pipeline_w2v = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            glove_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            explodeVectors,
            label_stringIdx])

nlp_model_w2v = nlp_pipeline_w2v.fit(train_sparkdf)

processed_w2v = nlp_model_w2v.transform(train_sparkdf)

processed_w2v.count()

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


2125

In [36]:
processed_w2v.select('description_combined','features','label').show()

+--------------------+--------------------+-----+
|description_combined|            features|label|
+--------------------+--------------------+-----+
|Aston Martin Lago...|[0.07989142090082...| 72.0|
|Fiat Chrysler Aut...|[-0.0646159574389...| 72.0|
|Ford Motor Compan...|[-0.0467863827943...| 72.0|
|General Motors is...|[-0.1721586585044...| 72.0|
| benefits, websit...|[-0.0966890454292...| 13.0|
| union, workers, ...|[0.13052451610565...| 13.0|
| cancer, research...|[-0.0404780320823...| 13.0|
| science, public,...|[0.07133471965789...| 13.0|
| chemistry, award...|[0.08529332280158...| 13.0|
| diabetes, overvi...|[0.01586429215967...| 13.0|
| bureau, advocacy...|[-0.0264652371406...| 13.0|
| afscme, union, w...|[0.09348104149103...| 13.0|
| randi, president...|[0.20875857770442...| 13.0|
|The American Psyc...|[0.09846050292253...| 13.0|
|This is the missi...|[0.03039588406682...| 13.0|
| health, union, r...|[0.04145178943872...| 13.0|
| scouts, girls, s...|[0.21835327148437...| 13.0|


In [37]:
(trainingData, testData) = processed_w2v.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1493
Test Dataset Count: 632


In [48]:

from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

testData = testData.where(num_nonzeros("features") != 0)

In [49]:
lrModel_w2v = lr.fit(trainingData)
predictions_w2v = lrModel_w2v.transform(testData)

predictions_w2v.select("description_combined","org_new_industry","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+------------------------------+------------------------------+-----+----------+
|          description_combined|              org_new_industry|                   probability|label|prediction|
+------------------------------+------------------------------+------------------------------+-----+----------+
| sefcu, online, tuesday, we...|Finance/Banking - Credit Union|[0.9911761776840043,3.52518...| 24.0|       0.0|
|Infor provides enterprise s...|SAAS - Leisure, Sports and ...|[0.7210925533063297,2.10307...| 57.0|       0.0|
|MedPro Healthcare Staffing ...|Corporate Services - Consul...|[0.6544045096437753,0.00702...|  0.0|       0.0|
|Modern Health is a mental w...|Leisure, Sports and Recreat...|[0.6129711349705067,0.01035...| 38.0|       0.0|
| created, public, insights,...|Energy and Environmental - ...|[0.5604486377860693,0.00738...| 36.0|       0.0|
| insights, digital, energy,...|Corporate Services - Consul...|[0.557076343128009,0.008085...|  0.0|    

In [50]:
#Accuracy
y_true = predictions_w2v.select("label")
y_true = y_true.toPandas()

y_pred = predictions_w2v.select("prediction")
y_pred = y_pred.toPandas()
print(accuracy_score(y_true.label, y_pred.prediction))

0.33134920634920634
